In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [12]:
Discrete(3).sample() #Used to create Space

1

In [14]:
Box(0,1,shape=(3,3)).sample()

array([[0.5713243 , 0.11105458, 0.11222368],
       [0.09095743, 0.9781431 , 0.18303299],
       [0.89597297, 0.67576754, 0.7503064 ]], dtype=float32)

In [16]:
Dict({'height':Discrete(2), "speed":Box(0,100,shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([30.148188], dtype=float32))])

In [17]:
MultiBinary(4) #Used to create This space

MultiBinary(4)

In [18]:
MultiBinary(4).sample()

array([0, 1, 0, 1], dtype=int8)

In [19]:
MultiDiscrete([5,2,2])#Used to make space

MultiDiscrete([5 2 2])

In [22]:
MultiDiscrete([5,2,2]).sample()

array([4, 1, 1], dtype=int64)

In [55]:
# class ShowerEnv(Env):
#     def __init__(self):
#         self.action_space = Discrete(3)
#         self.observation_space = Box(low=np.array([0]), high=np.array([100]))
#         self.state = 38 + random.randint(-3,3)
#         self.shower_length = 60
    
        
#     def step(self,action):
#         self.state += action - 1
#         self.shower_length -= 1
        
#         if self.state >= 37and self.state<=39:
#             reward = 1
#         else:
#             reward = -1
            
#         if shower.length <= 0:
#             done = True
#         else:
#             done = False
            
#         info={}
        
#         return self.state,reward,done,info
#     def render(self):
#         pass
#     def reset(self):
#         self.state = np.array([38+random.randint(-3,3)]).astype(float)
#         self.shower_length = 60
#         return self.state
        
    
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [56]:
env = ShowerEnv()

In [57]:
env.observation_space.sample()

array([5.4359684], dtype=float32)

In [58]:
env.action_space.sample()

0

In [63]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()  # Render the environment without specifying a render mode
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)  # Unpack values directly here
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


Episode:1 Score:-18
Episode:2 Score:14
Episode:3 Score:-26
Episode:4 Score:-20
Episode:5 Score:-26


In [64]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy',env,verbose = 1,tensorboard_log = log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [67]:
model.learn(total_timesteps = 40000)

Logging to Training\Logs\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -31.5    |
| time/              |          |
|    fps             | 702      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -31.2       |
| time/                   |             |
|    fps                  | 459         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006886013 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.1        |
|    explained_variance   | -5.84e-06   |

In [74]:
shower_path = os.path.abspath(os.path.join('Training', 'Saved Models', 'shower_PPO_model'))


In [75]:
model.save(shower_path)

In [76]:
del model

In [77]:
model = PPO.load(shower_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [78]:
evaluate_policy(model,env,n_eval_episodes = 10,render = True)

C:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
C:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(24.0, 54.99090833947008)